In [1]:
import sys
from pathlib import Path

import yaml

sys.path.insert(0, "..")

In [2]:
# If a util function was modified, use this cell to reload it without having to restart the kernel
%run ../runner/utils.py

## Setup benchmark campaign

### 20262002 Test PyPSA-DE MILP benchmarks for scaling

In [3]:
import subprocess

bucket_path = "gs://solver-benchmarks/tests/pypsa-de-milp-scaling/"

result = subprocess.run(
    ["gsutil", "ls", bucket_path],
    capture_output=True,
    text=True,
)

urls = [
    line.strip().replace("gs://", "https://storage.googleapis.com/")
    for line in result.stdout.strip().split("\n")
    if line.strip().endswith(".mps.gz")
]

print("Found files:", len(urls))
urls[:5]

Found files: 20


['https://storage.googleapis.com/solver-benchmarks/tests/pypsa-de-milp-scaling/pypsa-de-elec-uc_10_1h.mps.gz',
 'https://storage.googleapis.com/solver-benchmarks/tests/pypsa-de-milp-scaling/pypsa-de-elec-uc_11_1h.mps.gz',
 'https://storage.googleapis.com/solver-benchmarks/tests/pypsa-de-milp-scaling/pypsa-de-elec-uc_12_1h.mps.gz',
 'https://storage.googleapis.com/solver-benchmarks/tests/pypsa-de-milp-scaling/pypsa-de-elec-uc_13_1h.mps.gz',
 'https://storage.googleapis.com/solver-benchmarks/tests/pypsa-de-milp-scaling/pypsa-de-elec-uc_14_1h.mps.gz']

In [ ]:
import gzip
import shutil

import highspy
import requests


def download_and_unzip(url, dest_dir):
    dest_dir.mkdir(exist_ok=True)
    gz_path = dest_dir / url.split("/")[-1]
    mps_path = gz_path.with_suffix("")

    if not mps_path.exists():
        if not gz_path.exists():
            print(f"Downloading {url}")
            r = requests.get(url, stream=True)
            r.raise_for_status()
            with open(gz_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)

        print(f"Unzipping {gz_path}")
        with gzip.open(gz_path, "rb") as f_in:
            with open(mps_path, "wb") as f_out:
                shutil.copyfileobj(f_in, f_out)

    return mps_path


def get_num_variables(mps_path):
    h = highspy.Highs()
    h.readModel(str(mps_path))
    return h.getNumCol()


def classify_size(n_vars):
    if n_vars < 1e4:
        return "S"
    elif n_vars < 1e6:
        return "M"
    else:
        return "L"


benchmarks = {}
tmp_dir = Path("tmp_mps")
tmp_dir.mkdir(exist_ok=True)

for url in urls:
    filename = url.split("/")[-1]
    benchmark_base = filename.split("-")[0]

    mps_path = download_and_unzip(url, tmp_dir)
    n_vars = get_num_variables(mps_path)
    size_category = classify_size(n_vars)

    size_data = {
        "Name": filename.replace(".mps.gz", ""),
        "Size": size_category,
        "URL": url,
    }

    benchmarks.setdefault(benchmark_base, {"Sizes": []})
    benchmarks[benchmark_base]["Sizes"].append(size_data)

print("Classification complete.")

In [4]:
benchmarks = {}

for url in urls:
    filename = url.split("/")[-1].replace(".lp", "")
    parts = filename.split("-")

    benchmark_base = "-".join(parts[:-1])
    size_name = parts[-1]

    size_data = {
        "Name": size_name,
        "Size": None,
        "URL": url,
    }

    benchmarks.setdefault(benchmark_base, {"Sizes": []})
    benchmarks[benchmark_base]["Sizes"].append(size_data)

In [5]:
SOLVERS = [
    "highs",
    "gurobi",
    "scip",
]

In [6]:
from pathlib import Path

output_yaml = {
    "machine-type": "c4-highmem-8",
    "zone": "us-central1-a",
    "years": [2025],
    "solver": " ".join(SOLVERS),
    "benchmarks": benchmarks,
}

out_dir = Path("../infrastructure/benchmarks/20260220-pypsa-de-milp-scaling")
out_dir.mkdir(parents=True, exist_ok=True)

with open(out_dir / "00-campaign.yaml", "w") as f:
    yaml.dump(output_yaml, f, sort_keys=False)

print("Campaign YAML created.")

Campaign YAML created.


In [7]:
print("Total instances:", len(urls))
print("Total runs:", len(urls) * len(SOLVERS))

Total instances: 20
Total runs: 120
